In [19]:
import traceback
import subprocess
import os
import json
import requests
import pandas as pd
from loguru import logger
import numpy as np
from sklearn.decomposition import PCA

from typing import List, Dict

# Graph


In [20]:
class Step:
    def __init__(self,
                 step_id: str,
                 description: str,
                 dependencies: List[str],
                 input_vars: List[str],
                 output_vars: List[str],
                 additional_info):
        self.step_id = step_id
        self.desiption = description
        self.dependencies = dependencies
        self.input_vars = input_vars
        self.output_vars = output_vars
        self.additional_info = additional_info

    def __str__(self):
      return f"Step({repr(self.__dict__)})"

    def __repr__(self):
      return str(self)

# Language Modeling

In [21]:
class OpenAiLlamaApi:
    def __init__(self, api_url, api_key,  model_tag):
        self.api_url = api_url
        self.api_key = api_key
        self.model_tag = model_tag

    def execute_request(self, request):
        headers = {
            "Authorization": f"Bearer {self.api_key}",
            "Content-Type": "application/json"
        }
        data = {
            "model": self.model_tag,
            "messages": [{"role": "user", "content": request}]
        }
        response = requests.post(f"{self.api_url}/chat/completions", headers=headers, json=data)

        # Log the full response for debugging
        response_json = response.json()
        logger.info(f"Full API response: {response_json}")

        # Check if 'choices' key exists in the response
        if "choices" in response_json and response_json["choices"]:
            return response_json["choices"][0]["message"]["content"]
        else:
            logger.error(f"Request: {data}")
            logger.error(f"Response: {response_json}")
            raise ValueError("The response does not contain 'choices'. Full response: " + str(response_json))
class Model:
    def predict(self, promt):
        pass

class LlamaModel(Model):
    def __init__(self, llamaApi: OpenAiLlamaApi):
        self.api = llamaApi

    def predict_raw(self, prompt):
        return self.api.execute_request(prompt)

    def predict(self, prompt):
        generated_code = self.predict_raw(prompt)
        return self._clean_and_correct_code(generated_code)

    def _clean_and_correct_code(self, generated_code):
        cleaned_code = generated_code.replace("```python", "").replace("```", "").strip()
        cleaned_code_lines = [line for line in cleaned_code.split("\n") if not line.lower().startswith("here is the")]
        cleaned_code = "\n".join(cleaned_code_lines)
        return cleaned_code

class PromptGenerator:
    def __init__(self, example_script, dataset_info):
        self.example_script = example_script
        self.dataset_info = dataset_info

    def generate(self, step: Step, model_parameters):
        # f"Use these predefined parameters if needed: SizeSegment={SizeSegment}, gamma={gamma}, nu={nu}, kernel='{kernel}', NC_pca={NC_pca}, Dec_levels={Dec_levels}. "
        parameters_str = ', '.join(f'{name}={val}' for name, val in parameters.items())
        request = (
            f"Here is an example of a good step script:\n\n{self.example_script}\n\n"
            f"Here is an example of a good validation script:\n\n{self.example_script}\n\n"
            f"Write a Python function named 'step_{step.step_id}' for the following step: {step.desiption}. "
            f"The function should take {', '.join(step.input_vars)} as input and return {', '.join(step.output_vars)}. "
            f"Ensure to include necessary imports and handle edge cases. "
            f"Additional information: {step.additional_info}\n"
            f"The dataset has the following columns: {self.dataset_info['columns']}. "
            f"The data types are: {self.dataset_info['types']}. "
            f"Here's a sample of the data: {self.dataset_info['sample_data']}. "
            f"Value counts (top 5): {self.dataset_info['value_counts']}. "
            f"Statistical description: {self.dataset_info['description']}. "
            f"Use these predefined parameters if needed: {parameters_str}. "
            f"The input 'Segments' is a list of 1D numpy arrays, each representing a segment of the signal data. "
            f"Each segment should be normalized independently using sklearn's MinMaxScaler. "
            f"The output 'Segments_normalized' should be a list of normalized 1D numpy arrays. "
            f"Only return the function definition without any additional code or explanations."
        )
        return request

# Utils


In [22]:
def get_dataset_info(df):
    columns = df.columns.tolist()
    types = df.dtypes.apply(lambda x: str(x)).to_dict()
    sample_data = df.head().to_dict(orient='list')
    value_counts = {col: df[col].value_counts().head().to_dict() for col in df.columns}
    description = df.describe().to_dict()

    dataset_info = {
        'columns': columns,
        'types': types,
        'sample_data': sample_data,
        'value_counts': value_counts,
        'description': description
    }
    return dataset_info

# Code Generation

In [23]:
class CodeGenerator:
  def generate_source(self, step, step_dependencies, parameters, additonal_lines):
    pass

class MainCodeGenerator(CodeGenerator):
    def generate_source(self, step: Step, step_dependencies, parameters, additonal_lines=[]):
        # TODO: implement
        pass

class ValidationCodeGenerator(CodeGenerator):
  def generate_source(self, step: Step, step_dependencies: List[Step], parameters, additonal_lines=[]):
      input_params = step.input_vars
      output_params = step.output_vars

      validation_code = "import pandas as pd\n"
      for dep in sorted(step_dependencies):
          validation_code += f"from step_{dep.step_id} import step_{dep.step_id}\n"
      validation_code += f"from step_{step.step_id} import step_{step.step_id}\n\n"

      validation_code += "\n"
      for name, value in parameters.items():
          validation_code += f'{name} = {value}\n'
      validation_code += "\n"

      validation_code += "def validate_step():\n"

      for dep in sorted(step_dependencies):
          dep_inputs = ", ".join(dep.input_vars)
          dep_outputs = ", ".join(dep.output_vars)
          validation_code += f"    {dep_outputs} = step_{dep.step_id}({dep_inputs})\n"

      input_values = ", ".join(input_params)
      output_values = ", ".join(output_params)
      validation_code += f"    {output_values} = step_{step.step_id}({input_values})\n"
      validation_code += f"    print({output_values})\n"

      validation_code += "\nif __name__ == '__main__':\n"
      validation_code += "    validate_step()\n"
      return validation_code

# Orchestration


In [24]:
def topological_sort(steps: List[Step]) -> List[Step]:
    used = set()
    order = []
    id_to_step = {step.step_id: step for step in steps}
    def dfs(step):
        nonlocal order
        nonlocal used
        nonlocal id_to_step
        if step.step_id in used:
            return
        used.add(step.step_id)
        for dep_id in step.dependencies:
            dfs(id_to_step[dep_id])
        order.append(step)
    for step in steps:
        dfs(step)
    return order

class Orchestrator:
    MAX_RETRIES = 10

    def __init__(
          self,
          model,
          prompt_generator,
          validation_generator,
          main_generator,
          working_dir,
        ):
        self.model = model
        self.prompt_generator = prompt_generator
        self.validation_generator = validation_generator
        self.main_generator = main_generator
        self.working_dir = working_dir


    def run_steps(self, steps: List[Step], parameters):
        os.makedirs(self.working_dir, exist_ok=True)

        id_to_step = {step.step_id: step for step in steps}
        steps_ordered = topological_sort(steps)

        for step in steps_ordered:
            try:
                code_snippet, _ = self.generate_step_file(step, parameters)
                dependencies = [id_to_step[step_id] for step_id in step.dependencies]
                _, validation_filename = self.generate_validation_file(step, dependencies, parameters)
                success, message = self.validate_unit_code(validation_filename)

                for _ in range(Orchestrator.MAX_RETRIES):
                    if success:
                        break
                    code_snippet, _ = self.fix_step_source(step, code_snippet, message)
                    success, message = self.validate_unit_code(validation_filename)
                if not success:
                    continue
            except Exception as e:
                logger.error(f"Error processing step {step.step_id}: {e}\n{traceback.format_exc()}\n")
                break
            # TODO: generate main

    def generate_step_file(self, step, parameters):
        prompt = self.prompt_generator.generate(step, parameters)
        code_snippet = self.model.predict(prompt)
        filename = f'step_{step.step_id}.py'
        with open(self.working_dir + '/' + filename, 'w') as f:
            f.write(code_snippet)
        print(f'generated step source for step {step.step_id}, filename: {filename}')
        print(code_snippet)
        print()
        return code_snippet, filename

    def generate_validation_file(self, step, dependencies, parameters):
        source = self.validation_generator.generate_source(step, dependencies, parameters)
        filename = f'validate_step_{step.step_id}.py'
        with open(self.working_dir + '/' + filename, 'w') as f:
            f.write(source)
        print(f'generated validation source for step {step.step_id}, filename: {filename}')
        print(source)
        print()
        return source, filename

    def fix_step_source(self, step, code_snippet, error_message):
      prompt = (
          f"The following code snippet encountered an error:\n\n{code_snippet}\n\n"
          f"Error message:\n{error_message}\n\n"
          f"Please fix the code snippet to resolve the error without providing any explanations or comments."
      )
      source = self.model.predict(prompt)
      filename = f'step_{step.step_id}.py'
      with open(self.working_dir + '/' + filename, 'w') as f:
          f.write(source)
      print(f'fixing step source for step {step.step_id}, filename: {filename}')
      print(source)
      print()
      return source, filename

    def validate_unit_code(self, filename):
        try:
            work_path = os.path.realpath(self.working_dir)
            print(f'running {filename} in {work_path}')
            result = subprocess.run(
                ["python", filename],
                capture_output=True,
                text=True,
                cwd=work_path
            )
            print(f'exit_code = {result.returncode}')
            if result.returncode != 0:
                raise Exception(result.stderr)
            return True, result.stdout
        except Exception as e:
            return False, str(e)

# Main

In [25]:
EXAMPLE_STEP_SCRIPT = """
import pandas as pd
import pywt
from sklearn.preprocessing import StandardScaler

def step_40(Segments_normalized, Dec_levels):
    Features = []
    for segment in Segments_normalized:
        coeffs = pywt.wavedec(segment, 'db4', level=Dec_levels)
        features = [coefficient.mean() for coefficient in coeffs]
        Features.append(features)
    return StandardScaler().fit_transform(Features)
"""

EXAMPLE_VALIDATION_SCRIPT = """
import pandas as pd
from step_10 import step_10
from step_20 import step_20
from step_30 import step_30
from step_40 import step_40

def validate_step():
    csv_path = '/path/to/your/csv/file.csv'
    raw_data = step_10(csv_path)
    Segments = step_20(raw_data, SizeSegment=512)
    Segments_normalized = step_30(Segments)
    Features = step_40(Segments_normalized, Dec_levels=5)
    print(Features)

if __name__ == '__main__':
    validate_step()
"""

In [26]:
steps = [
    Step(
        step_id="10",
        description="Import raw data from CSV and segment it",
        dependencies=[],
        input_vars=["csv_path", "SizeSegment"],
        output_vars=["Segments"],
        additional_info="Use pandas to read the CSV and create segments of size SizeSegment."
    ),
    Step(
        step_id="20",
        description="Normalize the segmented data using MinMaxScaler",
        dependencies=["10"],
        input_vars=["Segments"],
        output_vars=["Segments_normalized"],
        additional_info="Segments is a list of 1D numpy arrays. Each segment should be normalized independently."
    ),
    Step(
        step_id="30",
        description="Extract features using wavelet decomposition",
        dependencies=["20"],
        input_vars=["Segments_normalized", "Dec_levels"],
        output_vars=["Features"],
        additional_info="Use pywavelets (pywt) library with 'db3' wavelet and specified Dec_levels."
    ),
    Step(
        step_id="40",
        description="Apply PCA for dimension reduction",
        dependencies=["30"],
        input_vars=["Features", "NC_pca"],
        output_vars=["PCA_Features", "pca"],
        additional_info="Use sklearn's PCA. Return both the transformed features and the PCA object."
    ),
    Step(step_id="50",
        description="Train model, evaluate, and calculate metrics",
        dependencies=["40"],
        input_vars=["PCA_Features", "kernel", "nu", "gamma"],
        output_vars=["FittedClassifier", "Prec_learn", "Prec_test"],
        additional_info="""
        1. Create labels: np.ones for learning data.
        2. Split data into train and test sets (80% train, 20% test).
        3. Create and fit a One-Class SVM classifier using sklearn.
        4. Predict labels for training data.
        5. Calculate error rate for training data.
        6. Predict labels for test data (assume all test data as anomaly, i.e., -1).
        7. Calculate error rate for test data.
        8. Calculate precision as 1 - error_rate for both training and test.
        Return the fitted classifier and both precision values.
        """
    )
]

In [27]:
with open('env.json', 'r') as f:
  credentials_dict = json.load(f)

API_URL = "https://openrouter.ai/api/v1"
API_KEY = credentials_dict["OPENROUTER_API_KEY"]
MODEL_TAG = "meta-llama/llama-3-70b-instruct"

df = pd.read_csv(
    '/Users/ilya/Desktop/GitHub_Repositories/Thesis/datasets/complicated_case/learning-file_1.csv')
dataset_info = get_dataset_info(df)

# Predefined parameters
raw_data = pd.read_csv(
    "/Users/ilya/Desktop/GitHub_Repositories/Thesis/datasets/complicated_case/learning-file_1.csv")
# Assume raw_data is a pandas DataFrame with 'timestamp' and 'signal' columns
signal_data = raw_data['signal'].values

# Adjust based on data size
SizeSegment = min(512, len(signal_data) // 100)
gamma = 'scale'  # Let sklearn choose an appropriate scale
nu = 0.1  # This might need domain knowledge to set appropriately
kernel = "rbf"  # This is often a good default

# PCA
# We'll use the signal data for PCA parameter calculation
pca = PCA().fit(signal_data.reshape(-1, 1))
cumulative_variance_ratio = np.cumsum(pca.explained_variance_ratio_)
NC_pca = np.argmax(cumulative_variance_ratio >= 0.95) + 1

Dec_levels = int(np.log2(SizeSegment)) - 3  # Adjust based on segment size
csv_path = '/Users/ilya/Desktop/GitHub_Repositories/Thesis/datasets/complicated_case/learning-file_1.csv'

parameters = {
    'csv_path': f"'{csv_path}'",
    "SizeSegment": f"{SizeSegment}",
    "gamma": f"'{gamma}'",
    "nu": f"{nu}",
    "kernel" : f"'{kernel}'",
    "NC_pca": f"{NC_pca}",
    "Dec_levels": f"{Dec_levels}",
}

In [28]:
llama_api = OpenAiLlamaApi(API_URL, API_KEY, MODEL_TAG)
model = LlamaModel(llama_api)
prompt_generator = PromptGenerator(EXAMPLE_STEP_SCRIPT, dataset_info)
validation_code_genrator = ValidationCodeGenerator()
main_code_generator = MainCodeGenerator()

orchestrator = Orchestrator(
    model,
    prompt_generator,
    validation_code_genrator,
    main_code_generator,
    'out'
)

In [29]:
orchestrator.run_steps(steps, parameters)

2024-07-31 19:56:35.081 | INFO     | __main__:execute_request:20 - Full API response: {'id': 'gen-44YNUpF2UuTf64cN3LYOcHRcBQVn', 'model': 'meta-llama/llama-3-70b-instruct', 'object': 'chat.completion', 'created': 1722441387, 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': "Here is the Python function named 'step_10':\n\n```\nimport pandas as pd\nimport numpy as np\nfrom sklearn.preprocessing import MinMaxScaler\n\ndef step_10(csv_path, SizeSegment):\n    data = pd.read_csv(csv_path)\n    signal = data['signal'].values\n    Segments = [signal[i:i+SizeSegment] for i in range(0, len(signal), SizeSegment)]\n    Segments_normalized = [MinMaxScaler().fit_transform(segment.reshape(-1, 1)).flatten() for segment in Segments]\n    return Segments_normalized\n```"}, 'finish_reason': 'stop', 'logprobs': None}], 'usage': {'prompt_tokens': 721, 'completion_tokens': 118, 'total_tokens': 839}}


generated step source for step 10, filename: step_10.py


import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

def step_10(csv_path, SizeSegment):
    data = pd.read_csv(csv_path)
    signal = data['signal'].values
    Segments = [signal[i:i+SizeSegment] for i in range(0, len(signal), SizeSegment)]
    Segments_normalized = [MinMaxScaler().fit_transform(segment.reshape(-1, 1)).flatten() for segment in Segments]
    return Segments_normalized

generated validation source for step 10, filename: validate_step_10.py
import pandas as pd
from step_10 import step_10


csv_path = '/Users/ilya/Desktop/GitHub_Repositories/Thesis/datasets/complicated_case/learning-file_1.csv'
SizeSegment = 307
gamma = 'scale'
nu = 0.1
kernel = 'rbf'
NC_pca = 1
Dec_levels = 5

def validate_step():
    Segments = step_10(csv_path, SizeSegment)
    print(Segments)

if __name__ == '__main__':
    validate_step()


running validate_step_10.py in /Users/ilya/Desktop/CodeGeneration-main

2024-07-31 19:56:40.819 | INFO     | __main__:execute_request:20 - Full API response: {'id': 'gen-KIZizbFRqPuPhsxYQpAHVVMzAR7v', 'model': 'meta-llama/llama-3-70b-instruct', 'object': 'chat.completion', 'created': 1722441397, 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': 'Here is the Python function `step_20` for normalizing the segmented data using MinMaxScaler:\n```\nimport numpy as np\nfrom sklearn.preprocessing import MinMaxScaler\n\ndef step_20(Segments):\n    Segments_normalized = []\n    for segment in Segments:\n        scaler = MinMaxScaler()\n        normalized_segment = scaler.fit_transform(segment.reshape(-1, 1)).reshape(-1)\n        Segments_normalized.append(normalized_segment)\n    return Segments_normalized\n```'}, 'finish_reason': 'stop', 'logprobs': {'tokens': None, 'token_logprobs': None, 'top_logprobs': None, 'text_offset': None}}], 'usage': {'prompt_tokens': 724, 'completion_tokens': 100, 'total_tokens': 824}}


generated step source for step 20, filename: step_20.py

import numpy as np
from sklearn.preprocessing import MinMaxScaler

def step_20(Segments):
    Segments_normalized = []
    for segment in Segments:
        scaler = MinMaxScaler()
        normalized_segment = scaler.fit_transform(segment.reshape(-1, 1)).reshape(-1)
        Segments_normalized.append(normalized_segment)
    return Segments_normalized

generated validation source for step 20, filename: validate_step_20.py
import pandas as pd
from step_10 import step_10
from step_20 import step_20


csv_path = '/Users/ilya/Desktop/GitHub_Repositories/Thesis/datasets/complicated_case/learning-file_1.csv'
SizeSegment = 307
gamma = 'scale'
nu = 0.1
kernel = 'rbf'
NC_pca = 1
Dec_levels = 5

def validate_step():
    Segments = step_10(csv_path, SizeSegment)
    Segments_normalized = step_20(Segments)
    print(Segments_normalized)

if __name__ == '__main__':
    validate_step()


running validate_step_20.py in /Users/ilya/Desktop/CodeGen

2024-07-31 19:56:43.995 | INFO     | __main__:execute_request:20 - Full API response: {'id': 'gen-SlD4ChYbWPyMhq5B1kua1PbE73oE', 'model': 'meta-llama/llama-3-70b-instruct', 'object': 'chat.completion', 'created': 1722441402, 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': "Here is the Python function `step_30`:\n```\nimport pandas as pd\nimport pywt\n\ndef step_30(Segments_normalized, Dec_levels):\n    Features = []\n    for segment in Segments_normalized:\n        coeffs = pywt.wavedec(segment, 'db3', level=int(Dec_levels))\n        features = [coefficient.mean() for coefficient in coeffs]\n        Features.append(features)\n    return Features\n```"}, 'finish_reason': 'eos', 'logprobs': None}], 'usage': {'prompt_tokens': 727, 'completion_tokens': 90, 'total_tokens': 817}}


...

2024-07-31 19:57:08.570 | INFO     | __main__:execute_request:20 - Full API response: {'id': 'gen-MVQ6JgpZR9wMrywpuZzKX4jkFy9X', 'model': 'meta-llama/llama-3-70b-instruct', 'object': 'chat.completion', 'created': 1722441423, 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': "Here is the corrected code snippet:\n\n```\nimport pandas as pd\nimport pywt\n\ndef step_20(Segments):\n    return Segments  # implementation of step_20 function\n\ndef step_30(Segments_normalized, Dec_levels):\n    Features = []\n    for segment in Segments_normalized:\n        coeffs = pywt.wavedec(segment, 'db3', level=int(Dec_levels))\n        features = [coefficient.mean() for coefficient in coeffs]\n        Features.append(features)\n    return Features\n\nSegments = [1, 2, 3, 4, 5]  # define Segments here\nDec_levels = 2  # define Dec_levels here\nSegments_normalized = step_20(Segments)\nFeatures = step_30(Segments_normalized, Dec_levels)\n```"}, 'finish_reason': 'stop', 'logprobs': {'toke

fixing step source for step 30, filename: step_30.py


import pandas as pd
import pywt

def step_20(Segments):
    return Segments  # implementation of step_20 function

def step_30(Segments_normalized, Dec_levels):
    Features = []
    for segment in Segments_normalized:
        coeffs = pywt.wavedec(segment, 'db3', level=int(Dec_levels))
        features = [coefficient.mean() for coefficient in coeffs]
        Features.append(features)
    return Features

Segments = [1, 2, 3, 4, 5]  # define Segments here
Dec_levels = 2  # define Dec_levels here
Segments_normalized = step_20(Segments)
Features = step_30(Segments_normalized, Dec_levels)

running validate_step_30.py in /Users/ilya/Desktop/CodeGeneration-main/Jupyter_notebook/out
exit_code = 1


2024-07-31 19:57:15.328 | INFO     | __main__:execute_request:20 - Full API response: {'id': 'gen-Fy8H72jv5o4IsUIIecv5nZaVqEED', 'model': 'meta-llama/llama-3-70b-instruct', 'object': 'chat.completion', 'created': 1722441430, 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': "Here is the corrected code snippet:\n\n```\nimport pandas as pd\nimport pywt\nimport numpy as np\n\ndef step_20(Segments):\n    return np.array(Segments)  # implementation of step_20 function\n\ndef step_30(Segments_normalized, Dec_levels):\n    Features = []\n    for segment in Segments_normalized:\n        coeffs = pywt.wavedec(segment, 'db3', level=int(Dec_levels))\n        features = [coefficient.mean() for coefficient in coeffs]\n        Features.append(features)\n    return Features\n\nSegments = [1, 2, 3, 4, 5]  # define Segments here\nDec_levels = 2  # define Dec_levels here\nSegments_normalized = step_20(Segments)\nFeatures = step_30(Segments_normalized, Dec_levels)\n```"}, 'finish_reaso

fixing step source for step 30, filename: step_30.py


import pandas as pd
import pywt
import numpy as np

def step_20(Segments):
    return np.array(Segments)  # implementation of step_20 function

def step_30(Segments_normalized, Dec_levels):
    Features = []
    for segment in Segments_normalized:
        coeffs = pywt.wavedec(segment, 'db3', level=int(Dec_levels))
        features = [coefficient.mean() for coefficient in coeffs]
        Features.append(features)
    return Features

Segments = [1, 2, 3, 4, 5]  # define Segments here
Dec_levels = 2  # define Dec_levels here
Segments_normalized = step_20(Segments)
Features = step_30(Segments_normalized, Dec_levels)

running validate_step_30.py in /Users/ilya/Desktop/CodeGeneration-main/Jupyter_notebook/out
exit_code = 1


KeyboardInterrupt: 